# Transitioning to Franklin from Raster Foundry

This notebook contains a walkthrough for interacting with Franklin, a [STAC API server](https://github.com/radiantearth/stac-api-spec) implementation, over HTTP.

The purpose of Franklin is to replace the data management and visualization capabilities that you currently rely on Raster Foundry for.

While with Raster Foundry you created [_projects_](../planet/rasterfoundry_create_project.py) and used cloud-optimized GeoTIFFs to create [_scenes_](../planet/rasterfoundry_register_scene.py), with Franklin you'll
create [_collections_](./create_collection.py) and use cloud-optimized GeoTIFFs to create [_items_](./create_item.py).

## Creating a collection

In Raster Foundry, a `project` was the entity responsible for associating several COGs so you could view a tile layer that they all contribute to. In Franklin, `collections` serve the same purpose.

To create a collection, you can run the `create_collection.py` script:

```bash
$ python create_collection.py --help
Usage: create_collection.py [OPTIONS] NAME

Options:
  -d, --description TEXT          Collection description. This should be a
                                  short punchy phrase that describes what
                                  kinds of items can be found in this
                                  collection
  --bbox BBOX                     Bounding box in comma-separated lower left
                                  x, lower left y, upper right x, upper right
                                  y format. This will expand as you add items,
                                  so picking a small bbox in the appropriate
                                  area makes sense to start.  [required]
  --start-date [%Y-%m-%d|%Y-%m-%dT%H:%M:%S|%Y-%m-%d %H:%M:%S]
                                  Datetime marking the beginning of coverage
                                  for this collection. Provide as an ISO 8601
                                  timestamp without a time zone -- it will be
                                  parsed as UTC. Defaults to now.  [required]
  --end-date [%Y-%m-%d|%Y-%m-%dT%H:%M:%S|%Y-%m-%d %H:%M:%S]
                                  Datetime marking the end of coverage for
                                  this collection. Provide as an ISO 8601
                                  timestamp without a time zone -- it will be
                                  parsed as UTC. Defaults to now.  [required]
  --license TEXT                  License of the underlying data. You can use
                                  identifiers from
                                  https://spdx.github.io/license-list-data/ in
                                  this field or the string "proprietary"
  --api-host TEXT                 The root of the STAC API you'd like to
                                  interact with. Defaults to localhost:9090
  --api-scheme TEXT               The scheme to use for API communication.
                                  Defaults to http
  --help                          Show this message and exit.
```

Creating a collection requires you to provide:

- `--start-date` and `--end-date` arguments. These arguments are formatted like ISO 8601 dates and define the [_temporal extent_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/collection-spec/collection-spec.md#temporal-extent-object) of the collection. To start, you can specify a relatively narrow temporal extent -- as you add items it will automatically expand.
- a `--bbox` argument. This argument defines the [_spatial extent_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/collection-spec/collection-spec.md#spatial-extent-object) of the collection. Similar to the temporal extent, it's sufficient to define a narrow area that you know you'll cover, since this will also expand as you add items to the collection.

Let's create a collection called `demo-collection` now. This collection will have a temporal extent covering July 2021 and a bbox covering a small point in the Canary Islands.

In [6]:
%%bash
python ../create_collection.py \
    demo-collection \
    --start-date 2021-07-01 \
    --end-date 2021-08-01 \
    --bbox -15.63,27.98,-15.60,27.99

Request json: {'type': 'Collection', 'id': 'demo-collection', 'stac_version': '1.0.0', 'description': 'A STAC collection for Agro Impacts', 'links': [], 'stac_extensions': [], 'title': 'demo-collection', 'extent': {'spatial': {'bbox': [[-15.63, 27.98, -15.6, 27.99]]}, 'temporal': {'interval': [['2021-07-01T00:00:00Z', '2021-08-01T00:00:00Z']]}}, 'license': 'proprietary'}
Created collection at /collections/demo-collection


After creating the collection, we can inspect it at `localhost:9090/collections/demo-collection`:

In [7]:
import requests

requests.get("http://localhost:9090/collections/demo-collection").json()

{'type': 'Collection',
 'stac_version': '1.0.0',
 'stac_extensions': [],
 'id': 'demo-collection',
 'title': 'demo-collection',
 'description': 'A STAC collection for Agro Impacts',
 'keywords': [],
 'license': 'proprietary',
 'providers': [],
 'extent': {'spatial': {'bbox': [[-17.072753906249986,
     0.0,
     1.0,
     28.94086176940554]]},
  'temporal': {'interval': [['2021-01-01T00:00:00Z',
     '2021-08-01T00:00:00Z']]}},
 'summaries': {},
 'properties': {},
 'links': [{'href': 'http://localhost:9090/collections/demo-collection/tiles',
   'rel': 'tiles',
   'type': 'application/json',
   'title': 'Tile URLs for Collection'},
  {'href': 'http://localhost:9090/collections/demo-collection',
   'rel': 'self',
   'type': 'application/json',
   'title': 'demo-collection'},
  {'href': 'http://localhost:9090', 'rel': 'root', 'type': 'application/json'},
  {'href': 'http://localhost:9090/collections/demo-collection/mosaic/d9b4db0f-7d80-4d96-b176-15a1400c01de',
   'rel': 'mosaic-definition

## Creating items

In Raster Foundry, you created _scenes_ with pointers to where the server could find a cloud-optimized GeoTIFF. In Franklin, we'll instead create [_items_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/item-spec.md) and store information about where to find the COG in the item's [_assets_](https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/item-spec.md#asset-object).

You create items with the `create_item.py` script:

```bash
$ python create_item.py --help
Usage: create_item.py [OPTIONS] ITEM_ID [%Y-%m-%d|%Y-%m-%dT%H:%M:%S|%Y-%m-%d
                      %H:%M:%S] COG_PATH COLLECTION_ID

Options:
  --eo-data PATH       Path to JSON containing information to fill in the EO
                       extension for this item. You can read more about the EO
                       extension at https://github.com/stac-
                       extensions/eo/tree/v1.0.0
  --cloud-cover FLOAT  Estimated cloud cover of the image
  --api-host TEXT      The root of the STAC API you'd like to interact with.
                       Defaults to localhost:9090
  --api-scheme TEXT    The scheme to use for API communication. Defaults to
                       http
  --help               Show this message and exit.
```

The required options for an item are an id, a date time, a path to a COG that you can read, and a collection to hold the item. While you could create a scene outside of a project in Raster Foundry, in Franklin, **you cannot create an item outside of a collection**.

Let's create two items, both of which point to COGs of Sentinel-2 imagery over the Canary Islands. You'll need to set an `AWS_PROFILE` environment variable to read the COGs linked here, but it doesn't matter which one.

In [10]:
%%bash
AWS_PROFILE=raster-foundry \
python ../create_item.py \
    demo-item-1 \
    2021-07-18 \
    s3://rasterfoundry-production-data-us-east-1/demo-cogs/s2-canary-islands-rgb-cog.tif \
    demo-collection

Created item at /collections/demo-collection/items/demo-item-1


In [12]:
%%bash
AWS_PROFILE=raster-foundry \
python ../create_item.py \
    demo-item-2 \
    2021-07-18 \
    s3://rasterfoundry-production-data-us-east-1/demo-cogs/s2-canary-islands-rgb-cog-2.tif \
    demo-collection

Created item at /collections/demo-collection/items/demo-item-2


After creating the items, we can verify that the collection knows about them by querying its `items` endpoint:

In [15]:
resp = requests.get("http://localhost:9090/collections/demo-collection/items").json()
resp

{'type': 'FeatureCollection',
 'features': [{'id': 'demo-item-1',
   'stac_version': '1.0.0',
   'stac_extensions': [],
   'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-15.007324218750007, 27.91673955951151],
      [-15.007324218750007, 28.94086176940554],
      [-13.86478258094437, 28.94086176940554],
      [-13.86478258094437, 27.91673955951151],
      [-15.007324218750007, 27.91673955951151]]]},
   'bbox': [-15.007324218750007,
    27.91673955951151,
    -13.86478258094437,
    28.94086176940554],
   'links': [{'href': 'http://localhost:9090/collections/demo-collection/items/demo-item-1/tiles',
     'rel': 'tiles',
     'type': 'application/json',
     'title': 'Tile URLs for Item'},
    {'href': 'http://localhost:9090/collections/demo-collection',
     'rel': 'parent',
     'type': 'application/json',
     'title': 'Parent collection'},
    {'href': 'http://localhost:9090',
     'rel': 'root',
     'type': 'application/json',
     'title': 'Welcome to

And if we're specifically interested in the `assets` each item has, we can extract that information from the response's `features` key:

In [16]:
[x["assets"] for x in resp["features"]]

[{'cog': {'href': 's3://rasterfoundry-production-data-us-east-1/demo-cogs/s2-canary-islands-rgb-cog.tif',
   'title': 'COG',
   'roles': ['data'],
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized'}},
 {'cog': {'href': 's3://rasterfoundry-production-data-us-east-1/demo-cogs/s2-canary-islands-rgb-cog-2.tif',
   'title': 'COG',
   'roles': ['data'],
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized'}}]

## Viewing tiles

Note that each of the items above has a key named `cog` in its `assets` property. Because items can have multiple assets, several of which can be tiled, Franklin requires one additional layer in order to produce tiles for a collection that wasn't required in Raster Foundry. This layer is a _mosaic definition_.

A _mosaic definition_ is a JSON document that describes which images you want to contribute to the resulting tiles and offers some metadata that could be useful for clients.

Here's an example document:

```json
{
    "id": "d9b4db0f-7d80-4d96-b176-15a1400c01de",
    "description": null,
    "center": [
        -16.16291,
        28.2916,
        14
    ],
    "items": [
        {
            "itemId": "demo-item-1",
            "assetName": "cog"
        },
        {
            "itemId": "demo-item-2",
            "assetName": "cog"
        }
    ],
    "minZoom": 2,
    "maxZoom": 30,
    "bounds": [
        -180,
        -90,
        180,
        90
    ]
}
```

The fields of a mosaic definition are:

- `id`: a UUID. You can generate a UUID with the `uuid4` method in the `uuid` python library.
- `description`: a textual description for humans. This information might be useful for clients to display information about what the layer represents.
- `center`: a three-element array containing the recommended central longitude, central latitude, and zoom level. The first two values are floats, while the third must be an integer.
- `items`: an array of `{"itemId": str, "assetName": str}` objects stating which assets in which items should contribute to the mosaic.
- `minZoom` / `maxZoom`: the minimum and maximum [zoom levels](https://wiki.openstreetmap.org/wiki/Zoom_levels) at which it makes sense to view these data.
- `bounds`: a bounding box describing where these tiles have coverage.

You associate mosaic definitions with collections. Every item in the mosaic definition must be in the collection, and at the time that you create the mosaic definition the items in question must have the assets listed in the `items` property.

You can create a mosaic definition with the `create_mosaic_definition.py` script: